In [1]:
#Se debe tener instalado Ollama de manera local para que corra
from langchain_ollama import ChatOllama

llm= ChatOllama(model="llama3:8b")

In [2]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="nomic-embed-text")

Ahora definiremos una manera de guardar la información de forma vectorial

In [3]:
#De manera local
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)
vector_store_fallos = InMemoryVectorStore(embeddings)

Ahora crearemos la base de datos en base a Ley_Consumidor.csv

In [4]:
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

def Cargar_documentos_csv(df):
    splitter = RecursiveCharacterTextSplitter( chunk_size = 1000, chunk_overlap=100, add_start_index=True)

    documents = []
    for index, columna in df.iterrows():
        Texto_articulo = columna['Texto_articulo']
        articulo = str(columna['Articulo'])
        
        if len(Texto_articulo) > 1000:
            splits = splitter.split_text(Texto_articulo)
            for split in splits:
                documents.append(
                    Document( page_content =split, metadata={'Articulo': articulo})
                )
        else:
            documents.append(
                Document( page_content =Texto_articulo, metadata= {'Articulo': articulo})
            )
    return documents

In [5]:
def Cargar_documentos_fallos_csv(df):
    splitter = RecursiveCharacterTextSplitter( chunk_size = 2000, chunk_overlap=200, add_start_index=True)

    documents = []
    for index, columna in df.iterrows():
        Texto_sentencia = columna['Texto_sentencia']
        Leyes_mencionadas = columna['Leyes_mencionadas']
        Rol = columna['Rol']
        Fecha_sentencia = str(columna['Fecha_Sentencia'])
        Corte_origen = columna['Corte de origen']
        
        
        splits = splitter.split_text(Texto_sentencia)
        
        for split in splits:
            if len(Texto_sentencia) > 1500:
                
                documents.append(
                     Document(
                         page_content = split,
                         metadata={
                         'Rol': Rol,
                            'Fecha Sentencia': Fecha_sentencia,
                            'Corte de origen': Corte_origen,
                            'Leyes mencionadas': Leyes_mencionadas
                        })
                )
            else:
                
                documents.append(
                Document(
                    page_content = Texto_sentencia,
                    metadata={
                        'Rol': Rol,
                        'Fecha Sentencia': Fecha_sentencia,
                        'Corte de origen': Corte_origen,
                        'Leyes mencionadas': Leyes_mencionadas
                    }
                    )
                )
    return documents

In [6]:
import pandas as pd

df = pd.read_csv("Ley_consumidor.csv")
df2 = pd.read_csv("Fallos_judiciales_ley_19.496.csv")

In [7]:
#Haremos split en chunks y cargaremos en documentos el DataFrame
docs_fallo = Cargar_documentos_fallos_csv(df2)
print(f"Total characters: {len(docs_fallo[12].page_content)}")

Total characters: 1639


In [8]:
docs = Cargar_documentos_csv(df)
print(f"Total characters: {len(docs[12].page_content)}")

Total characters: 501


Donde podemos ver que en la posicion 12 de docs encontramos el articulo N°2 ter del DataFrame

In [9]:
print(docs[12])

page_content='Las  Ley 21398 Art. 1 N° 2 D.O. 24.12.2021 normas contenidas en esta ley se interpretarán siempre en favor de los consumidores, de acuerdo con el principio pro consumidor, y, de manera complementaria, según las reglas contenidas en el párrafo 4° del Título Preliminar del Código Civil. TÍTU Ley N° 19.496 D.O. 07.03.1997 LO II      Disposiciones Ley N° 19.496 D.O. 07.03.1997  generales Párrafo 1º Ley N° 19.496 D.O. 07.03.1997      Los derechos Ley N° 19.496 D.O. 07.03.1997  y deberes del consumidor' metadata={'Articulo': 'Artículo 2 ter.-'}


Guardando los documentos

In [10]:
docs_fallo_ids = vector_store_fallos.add_documents(documents=docs_fallo)
print(docs_fallo_ids[:3])

['3d72028a-5674-494a-ac62-8a45599e3587', '14c90132-f194-4919-b430-d561321efff1', 'dab0270c-4417-427f-b8f8-b8903732ec15']


In [11]:
docs_ids = vector_store.add_documents(documents=docs)
print(docs_ids[:3])

['eece99a3-96cf-4418-85cf-6031f4869ad3', '4e84f97b-73ed-448e-8cde-41c4dd0639b7', '190cc92a-f40e-406e-8f8f-5299708a95d7']


Ahora iniciaremos con el Retrieval and Generation (RG)

Definimos el Prompt

In [16]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Redacta una denuncia para ser presentada.

A continuación tienes dos bloques de contexto que debes usar para elaborar la denuncia:

**Contexto legal** – Fragmentos de la Ley 19.496 sobre Protección de los Derechos de los Consumidores:
----------------
{context}
----------------

**Jurisprudencia relacionada** – Fallos judiciales previos que abordan situaciones similares:
----------------
{context_fallo}
----------------

**Pregunta del usuario:**
{question}

**Instrucciones para tu respuesta:**
- Usa un lenguaje claro, técnico pero entendible para una persona no experta.
- Cita explícitamente los artículos aplicables, agrupándolos según su propósito.
- Si hay un Fallo judicial que respalde el caso, menciónalo indicando el Rol, la Corte y la fecha.

**Respuesta:**
""")


Estado y funciones de RAG

In [13]:
from typing import TypedDict, List

class State(TypedDict):
    question: str
    context: List[Document]
    context_fallo: List[Document]
    answer: str

def retrieve(state: State):
    pregunta = state["question"]
    retrieved_docs = vector_store.similarity_search(pregunta)
    return {"context": retrieved_docs}

def retrieve_fallos(state: State):
    pregunta = state["question"]
    retrieved_docs_fallo = vector_store_fallos.similarity_search(pregunta)
    return {"context_fallo": retrieved_docs_fallo}

def generate(state: State):
    if not state["context"]:
        return {"answer": "No tengo suficiente contexto para responder."}
    if not state["context_fallo"]:
        print("No tengo un caso parecido en mi base de datos")
    
    # 🔁 Eliminar duplicados por contenido para Context
    unique_docs = []
    seen_contents = set()
    for doc in state["context"]:
        if doc.page_content not in seen_contents:
            unique_docs.append(doc)
            seen_contents.add(doc.page_content)

    # Unir contenido limpio para Context
    docs_content = "\n\n".join(doc.page_content for doc in unique_docs)
    print("\n docs_content:")
    print(docs_content)

    # 🔁 Eliminar duplicados por contenido para Context_fallo
    unique_docs_fallo = []
    seen_contents_fallo = set()
    for fallo in state["context_fallo"]:
        if fallo.page_content not in seen_contents_fallo:
            unique_docs_fallo.append(fallo)
            seen_contents_fallo.add(fallo.page_content)

    # Unir contenido limpio para Context_fallo
    docs_content_fallo = "\n\n".join(fallo.page_content for fallo in unique_docs_fallo)
    print("\n docs_content_fallo:")
    print(docs_content_fallo)

    
    final_prompt = prompt.invoke({
        "question": state["question"],
        "context": docs_content,
        "context_fallo": docs_content_fallo
    })

    response = llm.invoke(final_prompt)
    
    # Agregar metadata de artículos como fuentes
    articles = set()
    for doc in unique_docs:
        metadata = getattr(doc, "metadata", {})
        articulo = metadata.get("Articulo")
        if articulo:
            articles.add(articulo)
            
        print("Metadata:", metadata)  # debug
    
    sources = "\n\nArtículos utilizados como fuente:\n" + "\n".join(f"- {a}" for a in sorted(articles))

    metadata_fuentes_fallos = set()
    for fallo in unique_docs_fallo:
        metadata = getattr(fallo,"metadata", {})
        for k, v in metadata.items():
            metadata_fuentes_fallos.add(f"{k}: {v}")

        print("Metadata_fallos:",metadata_fuentes_fallos)
    sources_fallos = "\n\nMetadatos de fallos utilizados como fuente:\n" + "\n".join(f" - {a}" for a in sorted(metadata_fuentes_fallos))    
    
    return {"answer": f"{response.content.strip()}\n\n{sources.strip()}\n\n{sources_fallos.strip()}"}



In [17]:
state = {"question":"quiero hacer una denuncia contra CGE por los daños económicos que sufrí debido a prolongados cortes de luz, incluyendo la pérdida de alimentos y electrodomésticos, considerando que ya presenté reclamos sin recibir respuesta efectiva",
         "context": [],
         "context_fallo": [],
         "answer": ""
        }

# 2. Llamar a la función `generate()` para obtener la respuesta
state.update(retrieve(state))
state.update(retrieve_fallos(state))
state.update(generate(state))


print(state["answer"])


 docs_content:
La r Ley N° 21.081 Art. 1 N° 39 D.O. 13.09.2018 esolución que dé inicio al procedimiento, cuando haya sido dictada en virtud de una denuncia fundada de una asociación de consumidores, ordenará su participación, salvo manifestación en contrario de ésta en la misma denuncia.

dar cumplimiento a lo señalado en la respuesta del servicio de atención al cliente en el plazo de cinco días hábiles, contado desde la comunicación al consumidor.      En caso Ley N° 20.555 Art. 1 N° 6 D.O. 05.12.2011  de incumplimiento de las obligaciones indicadas en los dos incisos anteriores, el Servicio Nacional del Consumidor deberá denunciar al proveedor ante el juez de policía local competente, para que, si procediere, se le sancione con una multa de hasta cincuenta unidades tributarias mensuales, sin perjuicio del derecho del consumidor afectado para denunciar el incumplimiento de las obligaciones referidas.

las denuncias puede realizarse a título  Ley N° 21.081 Art. 1 N° 22 D.O. 13.09.2018